In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [2]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

In [3]:
from sklearn import metrics
from pandas_ml import ConfusionMatrix
from matplotlib import pyplot as plt
import itertools

In [26]:
dataframe = pd.read_csv("D:/DATATHON/Task_2_train.csv")

In [27]:
dataframe.head(1)

,Unnamed: 0,news_text,news_number,news_type
0,0,US bloggers banned from entering UK,111111112,non-propaganda


In [28]:
dataframe.columns =['sentence_id','news_text','news_number','news_type']

In [11]:
dataframe.head(5)

,sentence_id,news_text,news_number,news_type
0,0,US bloggers banned from entering UK,111111112,non-propaganda
1,1,NaN,111111112,non-propaganda
2,2,Two prominent US bloggers have been banned fro...,111111112,non-propaganda
3,3,NaN,111111112,non-propaganda
4,4,Pamela Geller and Robert Spencer co-founded an...,111111112,propaganda


In [30]:
df1 = dataframe.set_index(['sentence_id','news_number'])

In [85]:
df1.shape

(15170, 2)

In [31]:
df1.head(3)

,,news_text,news_type
sentence_id,news_number,,
0,111111112,US bloggers banned from entering UK,non-propaganda
1,111111112,NaN,non-propaganda
2,111111112,Two prominent US bloggers have been banned fro...,non-propaganda


In [32]:
df1.isnull().sum()

news_text    907
news_type      0
dtype: int64

In [33]:
df1['news_type'].value_counts()

non-propaganda    11230
propaganda         3940
Name: news_type, dtype: int64

In [34]:
df1.isna().sum()

news_text    907
news_type      0
dtype: int64

In [84]:
df1.shape

(15170, 2)

In [35]:
df=df1.dropna()

In [36]:
df['news_type'].value_counts()

non-propaganda    10325
propaganda         3938
Name: news_type, dtype: int64

In [37]:
df.isna().sum()

news_text    0
news_type    0
dtype: int64

In [38]:
y=df['news_type']
df2 = df.drop('news_type',axis=1)
df2.head(2)

,,news_text
sentence_id,news_number,
0,111111112,US bloggers banned from entering UK
2,111111112,Two prominent US bloggers have been banned fro...


In [39]:
X_train,X_test,y_train,y_test = train_test_split(df2['news_text'],y,test_size =0.3, random_state= 100)

In [40]:
#tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.8)

In [41]:
tfidf_train = tfidf_vectorizer.fit(X_train)
tfidf_train = tfidf_vectorizer.transform(X_train)
tfidf_test = tfidf_vectorizer.transform(X_test)


In [99]:
# print(tfidf_test.toarray())
# 

In [233]:
# linear_clf = PassiveAggressiveClassifier(max_iter=5,tol=1e-3,random_state=1)   # F1 : 0.509 
linear_clf = PassiveAggressiveClassifier(max_iter=9,tol = 0.0001,random_state=100) # 0.5119
# linear_clf = PassiveAggressiveClassifier(max_iter=90,tol = 0.0001,random_state=100) # 0.489
linear_clf.fit(tfidf_train, y_train)
pred = linear_clf.predict(tfidf_test)
score_pa_tfidf = metrics.accuracy_score(y_test, pred)
score_pa_tfidf = round(score_pa_tfidf,3)
print("accuracy:   %0.3f" % score_pa_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_pa = (2*P*R)/(P+R)
print(F1_pa)

accuracy:   0.733
0.5119453924914676


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:451: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [237]:
lr = LogisticRegression(n_jobs=1, C=1e3)
lr.fit(tfidf_train, y_train)
pred = lr.predict(tfidf_test)
score_lr_tfidf = metrics.accuracy_score(y_test, pred)
score_lr_tfidf = round(score_lr_tfidf,3)
print("accuracy:   %0.3f" % score_lr_tfidf)
# cm_lr_tfidf = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda'])
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_lr = (2*P*R)/(P+R)
print(F1_lr)

accuracy:   0.725
0.4871459694989107


In [51]:
clf = MultinomialNB()

clf.fit(tfidf_train, y_train)

pred = clf.predict(tfidf_test)
score_nb_tfidf = metrics.accuracy_score(y_test, pred)
score_nb_tfidf = round(score_nb_tfidf,3)
print("accuracy:   %0.3f" % score_nb_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_nb_tfidf = (2*P*R)/(P+R)
print(F1_nb_tfidf)

accuracy:   0.738
0.1290824261275272


In [44]:
rf = RandomForestClassifier(random_state=100)
rf.fit(tfidf_train, y_train)
pred = rf.predict(tfidf_test)
score_rf_tfidf = metrics.accuracy_score(y_test, pred)
score_rf_tfidf = round(score_rf_tfidf,3)
print("accuracy:   %0.3f" % score_rf_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_rf = (2*P*R)/(P+R)
print(F1_rf)

accuracy:   0.743
0.31910946196660483


In [45]:
dt = DecisionTreeClassifier(random_state=100)
dt.fit(tfidf_train, y_train)
pred = dt.predict(tfidf_test)
score_dt_tfidf = metrics.accuracy_score(y_test, pred)
score_dt_tfidf = round(score_dt_tfidf,3)
print("accuracy:   %0.3f" % score_dt_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_dt = (2*P*R)/(P+R)
print(F1_dt)

accuracy:   0.710
0.4274527867342239


In [46]:
ada = AdaBoostClassifier()
ada.fit(tfidf_train, y_train)
pred = ada.predict(tfidf_test)
score_ada_tfidf = metrics.accuracy_score(y_test, pred)
score_ada_tfidf = round(score_ada_tfidf,3)
print("accuracy:   %0.3f" % score_ada_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_ada = (2*P*R)/(P+R)
print(F1_ada)

accuracy:   0.729
0.23280423280423276


In [175]:
# mlp = MLPClassifier(hidden_layer_sizes=(5,5,4)) #0.45  (8,8,5)) # 0.48
mlp = MLPClassifier(hidden_layer_sizes=(10,10,5),activation='relu', solver='adam') # 0.48()
mlp.fit(tfidf_train, y_train)
pred = mlp.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
score_mlp_tfidf1=round(score,3)
#print(score_knn_tfidf)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_mlp = (2*P*R)/(P+R)
print(F1_mlp)

accuracy:   0.714
0.47935291613452535


In [48]:
knn = KNeighborsClassifier()
knn.fit(tfidf_train, y_train)
pred = knn.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
score_knn_tfidf=round(score,3)
#print(score_knn_tfidf)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_knn = (2*P*R)/(P+R)
print(F1_knn)

accuracy:   0.720
0.06396255850234009


In [150]:
svm1 = SVC(C=1.0,gamma=1, kernel='sigmoid', random_state= 50)
# svm1 = SVC(C=1.0,gamma=1, kernel='poly', random_state= 50)
svm1.fit(tfidf_train, y_train)
pred = svm1.predict(tfidf_test)
score = metrics.accuracy_score(y_test, pred)
score_svm_tfidf = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_svm_tf1 = (2*P*R)/(P+R)
print(F1_svm_tf1)

accuracy:   0.753
0.3592233009708738


In [127]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [130]:

text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, max_iter=5, random_state=42))])

text_clf_svm = text_clf_svm.fit(X_train, y_train)
predicted_svm = text_clf_svm.predict(X_test)
np.mean(predicted_svm == y_test)

0.723066136947885

In [170]:
# svm2 = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)
svm2 = SGDClassifier(loss='perceptron', penalty='l2',alpha=0.001, random_state=42, max_iter=5, tol=None) # 0.498
# svm2 = SGDClassifier(loss='perceptron', penalty='elasticnet',alpha=0.001, random_state=42, max_iter=5, tol=None)
svm2.fit(tfidf_train, y_train)
pred = svm2.predict(tfidf_test)
score1_svm_tfidf = metrics.accuracy_score(y_test, pred)
score_svm_tfidf2 = round(score1,3)
print("accuracy:   %0.3f" % score1_svm_tfidf)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_svm2_tf = (2*P*R)/(P+R)
print(F1_svm2_tf)

accuracy:   0.703
0.49842022116903634


In [238]:
acc2 = {'Passive Aggresive':score_pa_tfidf,'Logistic':score_lr_tfidf,'AdaBoost':score_ada_tfidf,'NB':score_nb_tfidf,
       'RF':score_rf_tfidf,'DT':score_dt_tfidf,'KNN':score_knn_tfidf,"MLP_NN":score_mlp_tfidf1,"SVM":score1_svm_tfidf}
F1_tfidf = {'Passive Aggresive':F1_pa,'Logistic':F1_lr,'AdaBoost':F1_ada,'NB':F1_nb_tfidf,
      'RF':F1_rf,'DT':F1_dt,'KNN':F1_knn,"MLP_NN":F1_mlp,"SVM":F1_svm2_tf}
acc_tfidf = pd.DataFrame([acc2,F1_tfidf])
acc_tfidf.index=['Model Accuracy_tfidf','F1 Score_tfidf']
acc_tfidf

,AdaBoost,DT,KNN,Logistic,MLP_NN,NB,Passive Aggresive,RF,SVM
Model Accuracy_tfidf,0.729000,0.710000,0.720000,0.725000,0.714000,0.738000,0.733000,0.743000,0.703202
F1 Score_tfidf,0.232804,0.427453,0.063963,0.487146,0.479353,0.129082,0.511945,0.319109,0.498420


In [ ]:
# tn,fp,fn,tp = confusion_matrix(test_dy,test_pred).ravel() 
# sensitivity = (tp/(tp+fn))*100 # recall
# specificity = (tn/(tn+fp))*100
# accuracy = ((tp+tn)/(tp+tn+fp+fn))*100

# tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
# R = (tp/(tp+fn))*100 # recall
# P = (tp/(tp+fp))*100 # precision
# F1 = (2*P*R)/(P+R)
# print(tn,fp,fn,tp,sensitivity,specificity, F1)

### Building Count Vectorizer

In [176]:
# Initialize the `count_vectorizer` 
count_vectorizer = CountVectorizer(stop_words='english')

In [177]:
count_train = count_vectorizer.fit(X_train)
count_train = count_vectorizer.transform(X_train)
count_test = count_vectorizer.transform(X_test)

In [178]:
## Fitting Naive Baye's Classifier for Multinomial Model
clf = MultinomialNB()
clf.fit(count_train, y_train)
pred = clf.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_nb = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_nb_cnt = (2*P*R)/(P+R)
print(F1_nb_cnt)

accuracy:   0.760
0.5031446540880502


In [257]:
## Fitting Passive Aggresive Classifier Model

linear_clf = PassiveAggressiveClassifier(max_iter=11,tol = 0.0001,random_state=100)

linear_clf.fit(count_train, y_train)
pred = linear_clf.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_pa = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_pa_cnt = (2*P*R)/(P+R)
print(F1_pa_cnt)

accuracy:   0.732
0.49427312775330395


C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:451: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [182]:
rf = RandomForestClassifier(random_state=100)
rf.fit(count_train, y_train)
pred = rf.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_rf = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_rf_cnt = (2*P*R)/(P+R)
print(F1_rf_cnt)

accuracy:   0.741
0.41167108753315645


In [183]:
dt = DecisionTreeClassifier(random_state=100,max_depth=2)
dt.fit(count_train, y_train)
pred = dt.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_dt = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_dt_cnt = (2*P*R)/(P+R)
print(F1_dt_cnt)

accuracy:   0.725
0.043938161106590726


In [261]:
lr = LogisticRegression(n_jobs=1, C=1e5, random_state=10)
lr.fit(count_train, y_train)
pred = lr.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_lr = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_lr_cnt = (2*P*R)/(P+R)
print(F1_lr_cnt)

accuracy:   0.745
0.5058930190389845


In [185]:
ada = AdaBoostClassifier()
ada.fit(count_train, y_train)
pred = ada.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_ada = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_ada_cnt = (2*P*R)/(P+R)
print(F1_ada_cnt)

accuracy:   0.734
0.2804801010739103


In [264]:
mlp = MLPClassifier(hidden_layer_sizes=(50,50,40)) # initially(554_0.46)  (9,9,5)) # 0.476
# mlp = MLPClassifier(hidden_layer_sizes=(19,19,5),activation='logistic') # 0.483
# mlp = MLPClassifier(hidden_layer_sizes=(19,19,5),activation='logistic')
mlp.fit(count_train, y_train)
pred = mlp.predict(count_test)
score= metrics.accuracy_score(y_test, pred)
score_cnt_mlp=round(score,3)
#print(score_knn_tfidf)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_mlp_cnt = (2*P*R)/(P+R)
print(F1_mlp_cnt)

accuracy:   0.727
0.47623318385650226


In [188]:
svm1 = SVC(C=1.0,gamma=1, kernel='sigmoid', random_state= 50)
svm1.fit(count_train, y_train)
pred = svm1.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_svm1 = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_svm_cnt = (2*P*R)/(P+R)
print(F1_svm_cnt)

accuracy:   0.678
0.3740354062641852


In [189]:
svm2 = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42,max_iter=5, tol=None)
svm2.fit(count_train, y_train)
pred = svm2.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_svm2 = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_svm2_cnt = (2*P*R)/(P+R)
print(F1_svm2_cnt)

accuracy:   0.761
0.38752249550089984


In [190]:
knn = KNeighborsClassifier()
knn.fit(count_train, y_train)
pred = knn.predict(count_test)
score = metrics.accuracy_score(y_test, pred)
score_cnt_knn = round(score,3)
print("accuracy:   %0.3f" % score)
tn,fp,fn,tp = metrics.confusion_matrix(y_test, pred, labels=['non-propaganda', 'propaganda']).ravel()
R = (tp/(tp+fn)) # recall/sensitivity
P = (tp/(tp+fp)) # precision
F1_knn_cnt = (2*P*R)/(P+R)
print(F1_knn_cnt)

accuracy:   0.718
0.13600572655690768


In [191]:
acc1 = {'Passive Aggresive':score_cnt_pa,'Logistic':score_cnt_lr,'AdaBoost':score_cnt_ada,'NB':score_cnt_nb,
       'RF':score_cnt_rf,'DT':score_cnt_dt,'KNN':score_cnt_knn,"MLP_NN":score_cnt_mlp,"SVM":score_cnt_svm1}

F1_cnt = {'Passive Aggresive':F1_pa_cnt,'Logistic':F1_lr_cnt,'AdaBoost':F1_ada_cnt,'NB':F1_nb_cnt,
          'RF':F1_rf_cnt,'DT':F1_dt_cnt,'KNN':F1_knn_cnt,"MLP_NN":F1_mlp_cnt,"SVM":F1_svm_cnt}
acc_cnt = pd.DataFrame([acc1,F1_cnt])
acc_cnt.index=['Model Accuracy_cnt','F1 Score_cnt']
acc_cnt

,AdaBoost,DT,KNN,Logistic,MLP_NN,NB,Passive Aggresive,RF,SVM
Model Accuracy_cnt,0.73400,0.725000,0.718000,0.760000,0.714000,0.760000,0.71700,0.741000,0.678000
F1 Score_cnt,0.28048,0.043938,0.136006,0.463224,0.469467,0.503145,0.48137,0.411671,0.374035


In [265]:
performance = acc_cnt
performance.append(acc_tfidf, ignore_index=False)

,AdaBoost,DT,KNN,Logistic,MLP_NN,NB,Passive Aggresive,RF,SVM
Model Accuracy_cnt,0.734000,0.725000,0.718000,0.760000,0.714000,0.760000,0.717000,0.741000,0.678000
F1 Score_cnt,0.280480,0.043938,0.136006,0.463224,0.469467,0.503145,0.481370,0.411671,0.374035
Model Accuracy_tfidf,0.729000,0.710000,0.720000,0.725000,0.714000,0.738000,0.733000,0.743000,0.703202
F1 Score_tfidf,0.232804,0.427453,0.063963,0.487146,0.479353,0.129082,0.511945,0.319109,0.498420
